In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.impute import KNNImputer

In [2]:
df = pd.read_csv("qm9.csv")

In [3]:

df

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv
0,gdb_1,C,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469
1,gdb_2,N,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,26.1563,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316
2,gdb_3,O,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,19.0002,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002
3,gdb_4,C#C,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574
4,gdb_5,C#N,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,48.7476,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133880,gdb_133881,C1C2C3C4C5OC14C5N23,3.59483,2.198990,1.904230,1.6637,69.37,-0.2254,0.0588,0.2842,760.7472,0.127406,-400.633868,-400.628599,-400.627654,-400.663098,23.658
133881,gdb_133882,C1N2C3C2C2C4OC12C34,3.65648,2.142370,1.904390,1.2976,69.52,-0.2393,0.0608,0.3002,762.6354,0.127495,-400.629713,-400.624444,-400.623500,-400.658942,23.697
133882,gdb_133883,C1N2C3C4C5C2C13CN45,3.67118,2.143140,1.895010,1.2480,73.60,-0.2233,0.0720,0.2953,780.3553,0.140458,-380.753918,-380.748619,-380.747675,-380.783148,23.972
133883,gdb_133884,C1N2C3C4C5CC13C2C45,3.52845,2.151310,1.865820,1.9576,77.40,-0.2122,0.0881,0.3003,803.1904,0.152222,-364.720374,-364.714974,-364.714030,-364.749650,24.796


In [4]:
random_state = 35

In [6]:
sampled_data = df.sample(n=20000, random_state=random_state)

In [8]:
print(sampled_data.head())

            mol_id              smiles        A        B        C      mu  \
80214    gdb_80215   OC1C2OC(=N)C12C#C  2.30911  1.61767  1.19613  2.3968   
38561    gdb_38562  C1C2OC3C4C=CC13C24  3.62779  2.04768  1.72903  1.3033   
128433  gdb_128434    COC(=N)N1C=NN=C1  4.59666  1.11509  0.90251  4.6009   
46046    gdb_46047    O=C1CC(CN1)N1CC1  4.51768  0.96556  0.82880  4.0801   
92628    gdb_92629       CC1OC2COCC2O1  2.61928  1.61948  1.55335  1.2958   

        alpha    homo    lumo     gap         r2      zpve          u0  \
80214   71.38 -0.2467 -0.0006  0.2461  1037.3102  0.098952 -436.608941   
38561   75.21 -0.2190  0.0061  0.2251   817.0529  0.138958 -384.614538   
128433  68.56 -0.2660 -0.0334  0.2325  1220.8171  0.113877 -450.138295   
46046   76.78 -0.2380  0.0362  0.2742  1365.3572  0.161073 -419.221121   
92628   70.36 -0.2408  0.0839  0.3247   980.1780  0.160888 -460.175337   

              u298        h298        g298      cv  
80214  -436.600959 -436.600015 -436.641

# Оставить только выбранные строки в исходном наборе данных

In [9]:
df = sampled_data

In [10]:
print(df.head())

            mol_id              smiles        A        B        C      mu  \
80214    gdb_80215   OC1C2OC(=N)C12C#C  2.30911  1.61767  1.19613  2.3968   
38561    gdb_38562  C1C2OC3C4C=CC13C24  3.62779  2.04768  1.72903  1.3033   
128433  gdb_128434    COC(=N)N1C=NN=C1  4.59666  1.11509  0.90251  4.6009   
46046    gdb_46047    O=C1CC(CN1)N1CC1  4.51768  0.96556  0.82880  4.0801   
92628    gdb_92629       CC1OC2COCC2O1  2.61928  1.61948  1.55335  1.2958   

        alpha    homo    lumo     gap         r2      zpve          u0  \
80214   71.38 -0.2467 -0.0006  0.2461  1037.3102  0.098952 -436.608941   
38561   75.21 -0.2190  0.0061  0.2251   817.0529  0.138958 -384.614538   
128433  68.56 -0.2660 -0.0334  0.2325  1220.8171  0.113877 -450.138295   
46046   76.78 -0.2380  0.0362  0.2742  1365.3572  0.161073 -419.221121   
92628   70.36 -0.2408  0.0839  0.3247   980.1780  0.160888 -460.175337   

              u298        h298        g298      cv  
80214  -436.600959 -436.600015 -436.641

In [11]:
df

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv
80214,gdb_80215,OC1C2OC(=N)C12C#C,2.30911,1.61767,1.19613,2.3968,71.38,-0.2467,-0.0006,0.2461,1037.3102,0.098952,-436.608941,-436.600959,-436.600015,-436.641349,30.459
38561,gdb_38562,C1C2OC3C4C=CC13C24,3.62779,2.04768,1.72903,1.3033,75.21,-0.2190,0.0061,0.2251,817.0529,0.138958,-384.614538,-384.608953,-384.608009,-384.643988,25.205
128433,gdb_128434,COC(=N)N1C=NN=C1,4.59666,1.11509,0.90251,4.6009,68.56,-0.2660,-0.0334,0.2325,1220.8171,0.113877,-450.138295,-450.130258,-450.129314,-450.172466,28.251
46046,gdb_46047,O=C1CC(CN1)N1CC1,4.51768,0.96556,0.82880,4.0801,76.78,-0.2380,0.0362,0.2742,1365.3572,0.161073,-419.221121,-419.212883,-419.211939,-419.254484,31.115
92628,gdb_92629,CC1OC2COCC2O1,2.61928,1.61948,1.55335,1.2958,70.36,-0.2408,0.0839,0.3247,980.1780,0.160888,-460.175337,-460.167671,-460.166727,-460.207765,29.594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104900,gdb_104901,CC1(C)NC1(CO)C#C,1.93991,1.64542,1.04339,2.6551,82.73,-0.2438,0.0245,0.2683,1209.0947,0.168243,-403.095486,-403.085447,-403.084503,-403.129637,38.090
53281,gdb_53282,CC#CC1CN1C(C)C,2.50825,0.95642,0.79844,1.5011,93.14,-0.2190,0.0420,0.2611,1571.3084,0.191229,-367.168954,-367.158033,-367.157088,-367.207375,38.770
72816,gdb_72817,CC12CCN1C(=O)OC2,2.69275,1.69957,1.30104,5.1536,71.11,-0.2533,0.0360,0.2893,1014.2399,0.149087,-439.119901,-439.112091,-439.111146,-439.152396,29.751
76944,gdb_76945,CC1N(C=O)C1(C)C=O,2.83987,1.10021,0.99865,1.6079,75.63,-0.2552,-0.0417,0.2135,1250.7317,0.144013,-439.096281,-439.086363,-439.085419,-439.131265,34.680


In [12]:
df.to_csv('qm9.csv', index=False)
df = pd.read_csv("qm9.csv")
df.to_csv('qm9_new.csv', index=False)
df.to_csv('qm9_new.csv', index=False)